## Step1：取得資料

In [1]:
# import date
import datetime
import pandas as pd
from pathlib import Path

d1 = datetime.date.today().strftime("%Y-%m-%d")
d1 = '2021-12-30'

In [2]:
df = pd.read_csv('./../crawler_data/clean_data/data_' + d1 + '.csv', sep=',')   
df_raw = df[['發文者','標題','推回文類別','內容','發文時間','觀看次數','推噓評價','發文者分數','URL','crawel_type']]
# df_raw.head(2)

In [3]:
# from bs4 import BeautifulSoup
import re

def pre_process(text):

  text = re.sub("\n", "", text)
  text = re.sub("https?:\/\/\S*", "", text)
  tokens = text.split()
  return " ".join(tokens)

# clean the text in Body column
df_raw['內容'] = df_raw['內容'].apply(pre_process)
df_raw['標題'] = df_raw['標題'].apply(pre_process)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [4]:
df['內容']

0     我今天幫客戶裝機的時候遇到一個很奇怪的問題\n\n中華電信給我的單子有以下資訊\nMask ...
1     請問目前各家電信，\n單純門號續約，還有500元以內的4G吃到飽不限速嗎？\n有沒有送市話的...
2     依照你的價格需求，\n目前應該只能先往台灣之星和亞太去找，\n關鍵字：新朋友399、238吃到飽。
3     在各電信收訊良好的情況\n可以考慮\n7-11的ibon mobile不限速(走遠傳線路)4...
4                                   企客方案有，看你的公司有沒有符合資格。
                            ...                        
66    問題是中華不用吃到飽 也沒什麼選擇 換到中華以外的業者就比較能精打細算5G速度越快 流量就越...
67    用慣不限速，改成量到降速會用的很痛苦，可以NP其他家不限速吃到飽，台哥699還包含D+2年會員可用
68                    你水管跟圖奇扣掉還有超過24G 我是建議不要啦你不如續599吃到飽
69    你網路用量大 可以辦5G 999 60GB到量還有4G 50M吃到飽購機優惠比較多！50M很...
70                                        我4g吃到飽一個月 快一T
Name: 內容, Length: 71, dtype: object

## Step2: 取得各家情緒分數

In [5]:
!pip install transformers #--use-feature=2020-resolver #requests beautifulsoup4 pandas numpy

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,DistilBertTokenizer, DistilBertModel
import torch
import transformers

In [7]:
pd.set_option('mode.chained_assignment', None)
df_raw.loc[:,'corpus'] = df['標題'].fillna(' ') + ": " + df['內容'].fillna(' ')

In [8]:
conditionFilter = 'and ~(corpus.str.contains("廣告"))'
conditionFET = 'corpus.str.contains("遠傳|FET") '
conditionCHT = 'corpus.str.contains("中華|CHT") '
conditionGT = 'corpus.str.contains("亞太|APGT") '
conditionTWN = 'corpus.str.contains("台哥大|台灣大|TWN") '
conditionTWNS = 'corpus.str.contains("台星|台灣之星") '

df_FET = df_raw.query(conditionFET + conditionFilter , engine='python')
df_CHT = df_raw.query(conditionCHT + conditionFilter , engine='python')
df_GT = df_raw.query(conditionGT + conditionFilter , engine='python')
df_TWN = df_raw.query(conditionTWN + conditionFilter , engine='python')
df_TWNS = df_raw.query(conditionTWNS + conditionFilter , engine='python')

In [9]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer.add_tokens(['遠傳','亞太','中華','台哥大','台灣大哥大','台星','台灣之星','5g','4g','5G','4G'], special_tokens=True)
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model.aux_logits = False
model.resize_token_embeddings(len(tokenizer))

Embedding(105890, 768)

In [10]:
df_raw.head(3)

,發文者,標題,推回文類別,內容,發文時間,觀看次數,推噓評價,發文者分數,URL,crawel_type,corpus
0,AntonyCyue,中華電信 IP設定疑問,主文,我今天幫客戶裝機的時候遇到一個很奇怪的問題中華電信給我的單子有以下資訊Mask 255.25...,2021-12-30 0:09,198.0,0.0,1分,https://www.mobile01.com/topicdetail.php?f=110...,Mobile01,中華電信 IP設定疑問: 我今天幫客戶裝機的時候遇到一個很奇怪的問題\n\n中華電信給我的單...
1,naturalman1979,門號續約4G吃到飽不限速,主文,請問目前各家電信，單純門號續約，還有500元以內的4G吃到飽不限速嗎？有沒有送市話的，謝謝！,2021-12-29 20:55,1452.0,3.0,97分,https://www.mobile01.com/topicdetail.php?f=18&...,Mobile01,門號續約4G吃到飽不限速: 請問目前各家電信，\n單純門號續約，還有500元以內的4G吃到飽...
2,老姆迷,門號續約4G吃到飽不限速,回文,依照你的價格需求，目前應該只能先往台灣之星和亞太去找，關鍵字：新朋友399、238吃到飽。,2021-12-29 21:30,0.0,0.0,126分,https://www.mobile01.com/topicdetail.php?f=18&...,Mobile01,門號續約4G吃到飽不限速: 依照你的價格需求，\n目前應該只能先往台灣之星和亞太去找，\n關...


In [11]:
def get_score(com_type, corpus, content):
    
    if content.find(com_type):
        corpus = content 
        #若內文有公司名稱,直接以內文分析, 標題不列入情緒分析
        #若內文沒有公司名稱, 則以主題+內文分析.
        #若只考慮內文,會有多筆都沒評分.
    
    tokens = tokenizer.encode(text=com_type, text_pair=corpus, 
                              return_tensors='pt', add_special_tokens = True)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

com_arr = ['FET','GT','CHT','TWN','TWNS']
com_name_arr = ['遠傳','亞太','中華','台哥大或台灣大哥大','台星或台灣之星']

for i, v in enumerate(com_arr) :
    df_raw['score'+v] = " "
    if len(locals()['df_'+v].index) :
        df_raw['score'+v] = pd.DataFrame(locals()['df_'+v]).apply(lambda r : get_score('對'+ com_name_arr[i] +'的看法:',
                                                                                       r['corpus'][:500], r['內容'][:500]), axis=1)


In [17]:
test = get_score('對遠傳的看法','好棒棒','')
test

4

In [16]:
test = get_score('對遠傳的看法','啊不就好棒棒','') #中文反話,需要透過FINE-TUNE調整.
test

4

In [18]:
pd.set_option("display.max_colwidth", 500)
df_raw[['標題','內容','scoreFET','scoreTWN','scoreCHT','scoreGT','scoreTWNS']].head(4)

,標題,內容,scoreFET,scoreTWN,scoreCHT,scoreGT,scoreTWNS
0,中華電信 IP設定疑問,我今天幫客戶裝機的時候遇到一個很奇怪的問題中華電信給我的單子有以下資訊Mask 255.255.255.248Gateway 192.168.11.172理論上我有 7個可用 IP ↓192.168.11.172192.168.11.173192.168.11.174192.168.11.175192.168.11.176192.168.11.177192.168.11.178首先 173 和 174 後端已經有設備在使用了，然後 175 不能用，數據機搜尋不到這個設備，我以為是設備故障，換了三台都一樣，後來我將設備的 IP 改為 171，居然可以使用，我太驚訝了，我很好奇為什麼，希望有大大能為我解惑....先在此感謝各位先進前輩~,NaN,NaN,2.0,NaN,NaN
1,門號續約4G吃到飽不限速,請問目前各家電信，單純門號續約，還有500元以內的4G吃到飽不限速嗎？有沒有送市話的，謝謝！,NaN,NaN,NaN,NaN,NaN
2,門號續約4G吃到飽不限速,依照你的價格需求，目前應該只能先往台灣之星和亞太去找，關鍵字：新朋友399、238吃到飽。,NaN,NaN,NaN,3.0,3.0
3,門號續約4G吃到飽不限速,在各電信收訊良好的情況可以考慮7-11的ibon mobile不限速(走遠傳線路)477元/月或無框行動不限速(走中華線路)466元/月(上述兩電信吃到飽不限速似乎好像有隱藏天花板，請自行查詢)或遠傳、台哥大的399電銷或亞太~12/31的238不限速吃到飽以上這些都低於500吃到飽另外，三大電信好像也有限速21M的吃到飽的方案，價錢大概都落在499元/月,3.0,3.0,3.0,3.0,NaN


In [13]:
df_final = df_raw[['發文者','標題','推回文類別','內容','發文時間','觀看次數','推噓評價','發文者分數','URL','crawel_type'
                   ,'scoreFET','scoreTWN','scoreCHT','scoreGT','scoreTWNS']]

In [14]:
df_final.to_csv('./../crawler_data/clean_data/data_' + d1 + '.csv', index = False)